# 01 - Humanized MOR model 10ns unbiased MD run data analysis

The goal of this project is to discover features of the $\mu$ opioid receptor and the ligands that bind to it that lead to specific types of agonism, such as biased agonism, partial agonism, or superagonism. 

We will start by loading the modules we will need to analyze our trajectory data, and collecting some of the files we will need throughout this notebook.

$$ \mathbf{F}=m\mathbf{\ddot{x}} $$

$$ \int_{0}^{\infty}f(\mathbf{x})\mathrm{d}x $$


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma
import pickle
from Bio.PDB import *
from Bio.PDB.MMCIFParser import MMCIFParser
parser = MMCIFParser()
import nglview
import mdtraj
import ipywidgets
from threading import Timer
from nglview.player import TrajectoryPlayer
pdbl = PDBList()
pdbl.retrieve_pdb_file('5C1M', pdir = '.', file_format = 'pdb')
parser = PDBParser(PERMISSIVE = True, QUIET = True) 
data = parser.get_structure("5c1m","pdb5c1m.ent")
pdbl.retrieve_pdb_file('6DDE', pdir = '.', file_format = 'pdb')
data1 = parser.get_structure("6dde","pdb6dde.ent")
pdbl.retrieve_pdb_file('6DDF', pdir = '.', file_format = 'pdb')
data2 = parser.get_structure("6ddf","pdb6ddf.ent")
pdbl.retrieve_pdb_file('4DKL', pdir = '.', file_format = 'pdb')
data3 = parser.get_structure("4dkl","pdb4dkl.ent")

Structure exists: './pdb5c1m.ent' 
Structure exists: './pdb6dde.ent' 
Structure exists: './pdb6ddf.ent' 
Structure exists: './pdb4dkl.ent' 


In [2]:
from IPython.display import display
def Fasta(seqdata=''):
    bundle = {}
    bundle['application/vnd.fasta.fasta'] = seqdata
    bundle['text/plain'] = seqdata
    display(bundle, raw=True)


In [3]:
with open('opioid_align.fasta') as f:
    seqdata = f.read()
Fasta(seqdata)

>P35372
------------------------------------------------------------
---------------------------------MDSSAAPTNASNCTDALAYSSCSPAPS
PGSWVNLSHLDGNLSDPCGPNRTDLGGRDSLCPPTGSPSMITAITIMALYSIVCVVGLFG
NFLVMYVIVRYTKMKTATNIYIFNLALADALAT----STLPFQSVNYLMGTWPFGTILCK
IVISIDYYNMFTSIFTLCTMSVDRYIAVCHPVKALDFRTPRNAKIINVCNWILSSAIGLP
VMFMATTKYRQGSIDCTLTF--SHPTWYWENLLKICVFIFAFIMPVLIITVCYGLMILRL
KSVRMLSGSKEKDRNLRRITRMVLVVVAVFIVCWTPIHIYVIIKALVTIPETTFQTVSWH
FCIALGYTNSCLNPVLYAFLDENFKRCFREFCIPTSSNIEQQNSTRIRQNTRDHPSTANT
VDRTNHQLENLEAETAPLP-----------------------------------------
--------
>P35372-2
------------------------------------------------------------
---------------------------------MDSSAAPTNASNCTDALAYSSCSPAPS
PGSWVNLSHLDGNLSDPCGPNRTDLGGRDSLCPPTGSPSMITAITIMALYSIVCVVGLFG
NFLVMYVIVRYTKMKTATNIYIFNLALADALAT----STLPFQSVNYLMGTWPFGTILCK
IVISIDYYNMFTSIFTLCTMSVDRYIAVCHPVKALDFRTPRNAKIINVCNWILSSAIGLP
VMFMATTKYRQGSIDCTLTF--SHPTWYWENLLKICVFIFAFIMPVLIITVCYGLMILRL
KSVRMLSGSKEKDRNLRRITRMVLVVVAVFIVCWTPIHIYVIIKALVTIPETTFQTVS

We will next go through the currently published structures (as of 2021_05_31) of the $\mu$ opioid receptor. The first of these structures is from

In [5]:
widget1 = nglview.show_biopython(data)
p = TrajectoryPlayer(widget1)
#widget1.add_ball_and_stick()
widget1


NGLWidget()

In [6]:
widget2 = nglview.show_biopython(data1)
p = TrajectoryPlayer(widget2)
#widget2.add_ball_and_stick()
widget2

NGLWidget()

In [7]:
widget3 = nglview.show_biopython(data2)
p = TrajectoryPlayer(widget3)
#widget3.add_ball_and_stick('PRO[224]')
#widget3.add_representation(repr_type="licorice", selection='227')
widget3

NGLWidget()

In [9]:
widget4 = nglview.show_biopython(data3)
p = TrajectoryPlayer(widget4)
#widget4.add_ball_and_stick()
widget4

NGLWidget()

We will explore some important important domains within the receptor that are thought to be key for bias. These regions serve as microwitches. The first we will examine the NPxxY motif.

In [10]:
widget5 = nglview.show_biopython(data2)
p = TrajectoryPlayer(widget5)
#widget3.add_ball_and_stick('PRO[224]')
widget5.add_representation(repr_type="licorice", selection='332')
widget5.add_representation(repr_type="licorice", selection='333')
widget5.add_representation(repr_type="licorice", selection='336')
widget5

NGLWidget()

The next we will consider is the PIF motif

In [11]:
widget6 = nglview.show_biopython(data2)
p = TrajectoryPlayer(widget6)
#widget3.add_ball_and_stick('PRO[224]')
widget6.add_representation(repr_type="licorice", selection='244')
widget6.add_representation(repr_type="licorice", selection='155')
widget6.add_representation(repr_type="licorice", selection='289')
widget6.add_representation(repr_type="licorice", selection='293')
widget6

NGLWidget()

In [10]:
pdb = '/home/njm3/Molecular_Dynamics/MOR_Work/step5_input.pdb'
trr = '/home/njm3/Molecular_Dynamics/MOR_Work/step7_1.trr'
print(pdb)

/home/njm3/Molecular_Dynamics/MOR_Work/step5_input.pdb


In [11]:
widget = nglview.show_mdtraj(mdtraj.load(pdb))
p = TrajectoryPlayer(widget)
widget.add_ball_and_stick()
p.spin = True
def stop_spin():
    p.spin = False
    widget.close()
#Timer(30, stop_spin).start()
widget

NGLWidget()

In [6]:
print(data.header["journal_reference"])

w.huang,a.manglik,a.j.venkatakrishnan,t.laeremans, e.n.feinberg,a.l.sanborn,h.e.kato,k.e.livingston, t.s.thorsen,r.c.kling,s.granier,p.gmeiner,s.m.husbandj.r.traynor,w.i.weis,j.steyaert,r.o.dror,b.k.kobilka structural insights into mu-opioid receptor activationature v. 524 315 2015 essn 1476-4687 26245379 10.1038/nature14886 


# 02 - todays date